In [17]:
import pandas as pd
import yfinance as yf
from datetime import datetime
from pandas import DataFrame

In [18]:
path_local = 'C:/Users/lucas/cotacaodeacoes/dados_acoes.parquet'

In [19]:
def obter_dados_acoes(acoes):
    # Obtém a data atual (utc now)
    data_atual = datetime.utcnow().date()

    # Obtém o primeiro dia do mês atual
    primeiro_dia_mes_atual = data_atual.replace(day=1)

    # Cria um DataFrame vazio para armazenar os dados
    dados_acoes = DataFrame(columns=["Ação", "Data", "Cotação", "Cotação Início do Mês", "Variação (%)"])

    # Loop para buscar dados de cada ação
    for acao in acoes:
        try:
            # Obtém os dados da ação usando a biblioteca Yahoo Finance
            acao_info = yf.Ticker(acao)

            # Obtém a cotação do dia atual
            cotação_atual = acao_info.history(period="1d")

            # Obtém a cotação no início do mês
            cotação_inicio_mes = acao_info.history(period="1d", start=primeiro_dia_mes_atual)

            # Calcula a variação em porcentagem
            variação_percentual = ((cotação_atual["Close"].iloc[0] - cotação_inicio_mes["Close"].iloc[0]) / cotação_inicio_mes["Close"].iloc[0]) * 100

            # Cria um DataFrame temporário com os dados da ação atual
            dados_temporarios = pd.DataFrame({"Ação": [acao], "Data": [data_atual], "Cotação": [cotação_atual["Close"].iloc[0]], "Cotação Início do Mês": [cotação_inicio_mes["Close"].iloc[0]], "Variação (%)": [variação_percentual]})

            # Concatena os dados temporários ao DataFrame principal
            dados_acoes = pd.concat([dados_acoes, dados_temporarios], ignore_index=True)

        except Exception as e:
            print(f"Erro ao obter dados para a ação {acao}: {str(e)}")

    return dados_acoes

In [ ]:
import smtplib
import email.message

def enviar_email(df_final):  
    corpo_email = """
    <p>f"{df_final}"</p>
    """

    msg = email.message.Message()
    msg['Subject'] = "Relatório Carteira"
    msg['From'] = 'lucasbonfim.pires@gmail.com'
    msg['To'] = 'lucasbonfim.pires@outlook.com'
    password = 'senha' 
    msg.add_header('Content-Type', 'text/html')
    msg.set_payload(corpo_email )

    s = smtplib.SMTP('smtp.gmail.com: 587')
    s.starttls()
    # Login Credentials for sending the mail
    s.login(msg['From'], password)
    s.sendmail(msg['From'], [msg['To']], msg.as_string().encode('utf-8'))
    print('Email enviado')



In [20]:
# Lista de ações da B3 que você deseja acompanhar com ".SA" no final
acoes = [
    'TGAR11.SA',
    'BPFF11.SA',
    'TRXF11.SA',
    'PVBI11.SA',
    'BTLG11.SA',
    'LVBI11.SA',
    'VILG11.SA',
    'IRDM11.SA',
    'MCCI11.SA',
    'MXRF11.SA',
    'RECR11.SA',
    'MALL11.SA',
    'XPML11.SA',
    'RBVA11.SA',
    'BBDC4.SA'
]

# Chama a função para obter os dados das ações
df_dados_das_acoes = obter_dados_acoes(acoes)

C:\Users\lucas\AppData\Local\Temp\ipykernel_236\4070649963.py:30: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dados_acoes = pd.concat([dados_acoes, dados_temporarios], ignore_index=True)


In [21]:
try:
  df_dado_acoes_passado = pd.read_parquet(f"{path_local}")
  df_merge =  pd.concat([df_dado_acoes_passado, df_dados_das_acoes])
  df_final = df_merge.drop_duplicates()
  df_final.to_parquet(f"{path_local}")
except:
  print('Primeira carga')
  df_final = df_dados_das_acoes
  df_final.to_parquet(f"{path_local}")


In [22]:
enviar_email(df_final)
resultado = pd.read_parquet(f"{path_local}")
resultado

,Ação,Data,Cotação,Cotação Início do Mês,Variação (%)
0,TGAR11.SA,2023-09-08,123.370003,121.949997,1.164416
1,BPFF11.SA,2023-09-08,72.910004,72.830002,0.109847
2,TRXF11.SA,2023-09-08,113.430000,113.000000,0.380531
3,PVBI11.SA,2023-09-08,104.389999,103.199997,1.153103
4,BTLG11.SA,2023-09-08,104.599998,104.290001,0.297246
5,LVBI11.SA,2023-09-08,118.980003,116.231377,2.364789
6,VILG11.SA,2023-09-08,106.639999,105.800003,0.793947
7,IRDM11.SA,2023-09-08,83.570000,82.500000,1.296969
8,MCCI11.SA,2023-09-08,95.760002,93.199997,2.746787
9,MXRF11.SA,2023-09-08,10.840000,10.760000,0.743494
